In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

# AI Basisklasse

Die folgende abstrakte Klasse `ChessAI` bildet die Basisklasse für alle AI-Versionen.
Sie enthält alle benötigten Methoden für das Eröffnungs- und Endspiel.
Die Methode für das Mittelspiel ist abstrakt und wird von der jeweiligen AI-Version implementiert.

Zunächst definieren wir einen Enum, um durch den folgenden Programmcode hinweg eine bessere Lesbarkeit zu gewährleisten.

In [ ]:
from enum import Enum

class State(Enum):
    OPENING_GAME = 0
    MIDDLE_GAME = 1
    END_GAME = 2
    FINISHED = 3

Nun wird die abstrakte Basisklasse erzeugt.

In [ ]:
from abc import ABC, abstractmethod
import chess
import random as random_module

class ChessAI(ABC):
    """Abstract base class for all AI versions"""
    random = random_module

    def __init__(self, player_name: str) -> None:
        super().__init__()
        self.state: State = State.OPENING_GAME
        self.name = player_name

    @abstractmethod
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Calculates the next move in the middle game."""
        pass

## Eröffnungsspiel

Für das Eröffnungsspiel wird das [Baron Polyglot Opening Book](https://www.chessprogramming.net/new-version-of-the-baron-v3-43-plus-the-barons-polyglot-opening-book/) von Richard Pijl verwendet. Hierbei handelt es sich um eine Sammlung von Eröffnungszügen im sog. 'polyglot book format'.

### Das Polyglot Book Format

Das Polyglot Book Format wird von H. G. Muller auf seiner [Website](http://hgm.nubati.net/book_format.html) sehr gut beschrieben. Die dortigen Ausführungen werden im Folgenden sehr komprimiert dargestellt.

Ein Polyglot Book besteht aus einer Serie von Einträgen, wobei ein Eintrag für einen Zug in einer bestimmten Board-Situation steht und jeweils 4 Attribute aufweist:

| Attribut | Beschreibung |
|---|---|
| key (64-bit unsigned int) | Der sog. Zobrist-Hash der exakten Board-Position. |
| move (16-bit unsigned int) | Die Angabe des Zuges (Quelle und Ziel als Zeile & Spalte) und ob eine Figur-Umwandlung stattfindet. |
| weight (16-bit unsigned int) | Die Qualität des Zuges, berechnet als $2 \cdot (gewonnene Spiele) + (verlorene Spiele)$, skaliert auf 16-bit. |
| learn (32-bit unsigned int) | Ein Feld, welches dafür benutzt werden kann, um Erfahrungen mit dem Zug zu speichern (bspw. wie oft und mit welchem Erfolg dieser Zug gespielt wurde). |

Die Python `chess` Bibliothek bietet für die Arbeit mit Polyglot Opening Books die Klasse [`chess.polyglot.MemoryMappedReader`](https://python-chess.readthedocs.io/en/latest/polyglot.html) an. Eine Instanz dieser Klasse kann mithilfe der Funktion `chess.polyglot.open_reader("Pfad_zur_Polyglot_Bibliotheksdatei")` erzeugt werden.

Die folgende Funktion gibt eine Instanz der eben genannten Klasse zurück. Falls notwendig wird die Instanz erstmalig erstellt.

In [ ]:
import chess.polyglot

# Definition of the opening book reader
class ChessAI(ChessAI):  # type: ignore
    def get_opening_game_reader(self: ChessAI) -> chess.polyglot.MemoryMappedReader:
        """Returns the opening game reader"""
        try:
            return self.opening_game_reader
        except AttributeError:
            self.opening_game_reader: chess.polyglot.MemoryMappedReader = chess.polyglot.open_reader("../lib/opening/Baron/baron30.bin")
            return self.opening_game_reader

Eine Instanz dieser Klasse bietet nun die folgende Funktion an:
- `get(board)`: Gibt für ein gegebenes Board den (ersten) Eintrag mit dem höchsten `weight` zurück.  
  Die gekürzte Funktionssignatur lautet `get(board: Union[chess.Board, int]) → Union[chess.polyglot.Entry, None]`

Diese folgende Funktion nimmt als Parameter das aktuelle Board und verwendet `get(board)` um den besten Zug während des Eröffnungsspiels zu ermitteln. Der ermittelte Zug wird zurückgegeben.

In [ ]:
class ChessAI(ChessAI):  # type: ignore
    def get_next_opening_game_move(self: ChessAI, board: chess.Board) -> chess.Move:
        """Picks the best game entry out of all opening book entries and returns its move."""
        moves = list(self.get_opening_game_reader().find_all(board))
        best_move_weight = max([entry.weight for entry in moves])
        best_moves = [entry for entry in moves if entry.weight == best_move_weight]
        return self.random.choice(best_moves).move

## Endspiel

Das Endspiel nutzt sogenannte *Syzygy tablebases*, die zu jedem möglichen Spielstand $ \mathcal{B} $ für bis zu 7 Figuren Informationen über die Metriken WDL (**Winn/Draw/Loss**) und DTZ (**Depth to Zero**) bereitstellen. Dabei sind `WDL` und `DTZ` Bewertungsfunktionen, die wie folgt definiert sind

$$ \texttt{WDL}: \mathcal{B} \rightarrow v $$

mit $ v \in \{ -2, -1, 0, 1, 2 \} $ und

$$ \texttt{DTZ}: \mathcal{B} \rightarrow w $$

mit $ w \in \{ -100, \dots, -1, 0, 1, \dots, 100 \} $.

### 50-Züge-Regel

Die 50-Züge-Regel beim Schach besagt, dass eine Partie dann als Remis gewertet werden kann, wenn in den letzten 50 aufeinanderfolgenden Spielzügen weder eine Figur geschlagen, noch ein Bauer gezogen wurde.

### WDL

In `WDL`-Dateien (Dateiendung `.rtbw`) sind Informationen zu Sieg, Remis und Niederlage unter Berücksichtigung der 50-Züge-Regel gespeichert. Auf diese Informationen kann während der Suche zugegriffen werden.

Dabei wird den Elementen der Menge $ \{ -2, -1, 0, 1, 2 \} $ die im Folgenden definierte Eigenschaft zugeordnet

* `2` die ziehende Seite gewinnt
* `0` es liegt ein Remis vor
* `-2` die ziehende Seite verliert
* `1` bei einem 'cursed win'
* `-1` bei einem 'blessed loss'

### DTZ
`DTZ`-Dateien (Dateiendung `.rtbz`) enthalten Informationen über die *Lagenzählung* für den Zugriff zu Beginn der Suche. Um Speicherplatz zu sparen gibt es in Anlehnung an die 50-Züge-Regel auch die Metrik `DTZ50`, die lediglich für eine Spielseite Informationen speichert. Für jede mögliche Position repräsentiert die DTZ die Anzahl der Züge des Gewinners bis zum Sieg. Hierfür werden folgende zwei Annahmen gemacht

* Der Gewinner minimiert die DTZ
* Der Verlierer maximiert die DTZ

### Zusammenhang von WDL und DTZ

Jedes Endspiel nutzt ein Paar dieser Informationen zur Evaluation des besten nächsten Zugs.

Nachstehende Tabelle beschreibt zusammengefasst das Verhalten der Funktionen.

| WDL | DTZ             | Beschreibung                                                                                                                                                                        |
| --- | --------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| -2  | -100 <= n <= -1 | Unbedingter Verlust (unter der Annahme, dass der 50-Züge-Zähler Null ist), wobei ein Nullzug in -n Zügen erzwungen werden kann.                                                     |
| -1  | n < -100        | Verlust, aber Remis nach der 50-Züge-Regel. Ein Nullzug kann in -n Zügen oder -n - 100 Zügen erzwungen werden (wenn eine spätere Phase für den 'blessed loss' verantwortlich ist).  |
|  0  | 0               | Remis.                                                                                                                                                                              |
|  1  | 100 < n         | Sieg, aber Unentschieden nach der 50-Züge-Regel. Ein Nullzug kann in n Zügen oder n - 100 Zügen erzwungen werden (wenn eine spätere Phase für den 'cursed win' verantwortlich ist). |
|  2  | 1 <= n <= 100   | Unbedingter Sieg (unter der Annahme, dass der 50-Züge-Zähler Null ist), wobei ein Nullzug in n Zügen erzwungen werden kann.                                                         |

Um den bestmöglichen nächsten Zug durch die Endspielbibliothek erhalten zu können, definieren wir den $ \texttt{Folge-Zug} $.

Ein Folge-Zug $ \mathcal{F} $ ist ein Tripel der Form

$$ \mathcal{F} = \langle M, W, D \rangle $$

wobei

- $ M $ ein Zug,
- $ W $ der der Position nach dem Zug $ M $ über die Metrik $ \texttt{WDL} $ zugeordnete Wert,
- $ D $ der der Position nach dem Zug $ M $ über die Metrik $ \texttt{DTZ} $ zugeordnete Wert ist.

Typ-Definition des Folge-Zugs:

In [ ]:
from typing import Tuple
class ChessAITypes():
    FollowingMove = Tuple[chess.Move, int, int]

Die Python `chess` Bibliothek bietet für die Arbeit mit Syzygy Endgame Tablebases die Klasse [`chess.syzygy.Tablebase`](https://python-chess.readthedocs.io/en/latest/syzygy.html) an. Eine Instanz dieser Klasse kann mithilfe der Funktion `chess.syzygy.open_tablebase("Pfad_zum_Syzygy_Tablebase_Ordner")` erzeugt werden.

Die folgende Funktion gibt eine Instanz der eben genannten Klasse zurück. Falls notwendig wird die Instanz erstmalig erstellt.

In [ ]:
import chess.syzygy

# Definition of the end game tablebase reader
class ChessAI(ChessAI):  # type: ignore
    def get_end_game_reader(self: ChessAI):
        try:
            return self.end_game_reader
        except AttributeError:
            self.end_game_reader: chess.syzygy.Tablebase = chess.syzygy.open_tablebase("../lib/ending/3-4-5")
            return self.end_game_reader

Die Funktion `get_end_game_following_moves` nimmt als Argument eine Liste von Zügen $ \textit{Moves} $, berechnet für diese iterativ die Metriken WDL und DTZ und gibt eine Liste von Folge-Zügen $ \mathcal{F} $ zurück.

Die Liste $ \mathcal{F} $ von Folge-Zügen wird benötigt um anschließend den besten Zug (anhand WDL und DTZ) auszuwählen.

In [ ]:
from typing import List
class ChessAI(ChessAI):  # type: ignore
    def get_end_game_following_moves(self: ChessAI, moves: List[chess.Move], board: chess.Board) -> List[ChessAITypes.FollowingMove]:
        """Gets WDL and DTZ from all given moves."""
        next_moves = []
        for move in moves:
            board.push(move)
            wdl_after_next_move = self.get_end_game_reader().probe_wdl(board)
            dtz_after_next_move = self.get_end_game_reader().probe_dtz(board)
            board.pop()
            next_moves.append((move, wdl_after_next_move, dtz_after_next_move))
        return next_moves

Die Funktion `get_end_game_relevant_moves` nimmt eine Liste von Folge-Zügen $ \mathcal{F} $ sowie eine WDL und gibt alle Züge, gefiltert nach dem Wert der WDL.

Idee:  
Es sind nur Züge relevant bei denen die jeweilige WDL des aktuellen Boards beibehalten wird (eine bessere WDL ist nicht möglich, eine schlechtere nicht erwünscht).

In [ ]:
class ChessAI(ChessAI):  # type: ignore
    def get_end_game_relevant_moves(self: ChessAI, moves: List[ChessAITypes.FollowingMove], wdl: int) -> List[ChessAITypes.FollowingMove]:
        """Filter for moves with given WDL and returns them."""
        return [ move for move in moves if move[1] == wdl ]

Die Funktion `get_end_game_best_zeroing_move` nimmt als Argument eine Liste von Folge-Zügen $ \mathcal{F} $ und prüft, ob einer dieser Züge die Metrik DTZ auf Null setzt, also ein Schlag- oder Bauernzug ist und gibt diesen zurück. Alternativ gibt diese Funktion den Zug mit der größten DTZ zurück.

Idee:  
Diese Methode wird aufgerufen wenn der aktuelle Spieler am gewinnen ist (WDL = 2) und somit die DTZ Null setzen möchte um ein Remis durch die 50-Züge-Regel zu verhindern. Da in diesem Fall die DTZ negativ ist, wird das Maximum gebildet.

In [ ]:
class ChessAI(ChessAI):  # type: ignore
    def get_end_game_best_zeroing_move(self: ChessAI, board: chess.Board, moves_tpl: List[ChessAITypes.FollowingMove]) -> ChessAITypes.FollowingMove:
        """Gets zeroing move if available or move with maximum DTZ."""
        for move_tpl in moves_tpl:
            if board.is_zeroing(move_tpl[0]):
                return move_tpl
        return max(moves_tpl, key=lambda move: move[2])

Die Funktion `get_next_end_game_move` berechnet mithilfe des aktuellen Boards und der zuvor vorgestellten Hilfsfunktionen den besten Zug für die aktuelle Seite und gibt diesen zurück. 

Hierbei werden folgende Fälle unterschieden:

1. Das Spiel ist beendet. In diesem Fall wird `None` zurückgegeben. 
2. Der Spieler gewinnt aktuell (WDL = 2), d. h. das Ziel ist es nun die DTZ auf Null zu setzen um ein Remis mithilfe der 50-Züge-Regel zu verhindern. Der diesbezüglich beste Zug wird gesucht und zurückgegeben.
3. Der Spieler verliert aktuell (WDL = -2), d. h. das Ziel ist es nun die DTZ zu maximieren um ein Remis mithilfe der 50-Züge-Regel zu erreichen. Der diesbezüglich beste Zug wird gesucht und zurückgegeben.
4. Das Spiel endet unentschieden (WDL = 0), die aktuelle DTZ ist 0. Hier wird ein zufälliger Zug mit derselben WDL (Remis) ausgewählt.

In [ ]:
class ChessAI(ChessAI):  # type: ignore
    def get_next_end_game_move(self: ChessAI, board: chess.Board) -> chess.Move | None:
        """Gets next endgame move based on WDL."""
        # Check if the game is over due to checkmate, stalemate, insufficient material, ...
        if board.is_game_over():
            return None

        # Get the WDL from the currently given board
        wdl = self.get_end_game_reader().probe_wdl(board)

        # Get list of metrics from all legal moves
        moves = self.get_end_game_following_moves(board.legal_moves, board)

        # Board.turn is winning
        if wdl == 2:
            # Get all relevant moves with the same WDL
            relevant_moves = self.get_end_game_relevant_moves(moves, -2)

            # Get best move as minimum from list of FollowingMove's with given DTZ as key
            best_move = self.get_end_game_best_zeroing_move(board, relevant_moves)[0]
        # Board.turn is losing:
        elif wdl == -2:
            # Select best move by filtering for max DTZ
            best_move = max(moves, key=lambda move: move[2])[0]
        # Board is a draw
        elif wdl == 0:
            # Select move by filtering for WDL
            relevant_moves = self.get_end_game_relevant_moves(moves, 0)
            best_move = relevant_moves[0][0]
        else:
            # Some error occurs
            assert False, f'wdl has unknown value in get_next_end_game_move: wdl = {wdl}'

        return best_move

## Berechnung des nächsten Zuges

Diese Funktion nimmt als Argument das aktuelle Board und gibt den nächsten Spielzug, sowie den neuen Spielstatus (State) zurück.

Folgende Fallunterscheidung wird (in dieser Reihenfolge) durchgeführt:

1. Falls die 50-Züge-Regel greift oder eine andere Endbedingung (siehe Hinweis) zutrifft, ist das Spiel beendet. Somit wird kein Zug (`None`) und der Status `FINISHED` zurückgegeben.
2. Falls sich das Board aktuell im Eröffnungsspiel befindet, und ein Zug aus der Polyglot-Eröffnungsbibliothek vorhanden ist, wird dieser zusammen mit dem Status `OPENING_GAME` zurückgegeben.
3. Falls weniger als 6 Spieler vorhanden sind, wird der nächste Spielzug mithilfe der Endspiel-Bibliothek ermittelt und zusammen mit dem Spielstatus `END_GAME` zurückgegeben.
    - Sollte die Endspiel-Bibliothek **keinen** gültigen Zug finden, ist das Spiel beendet und es wird `None` mit dem Status `FINISHED` zurückgegeben.
4. Sind die vorherigen Fälle nicht anwendbar, so befindet sich das Board im Mittelspiel und der nächste Zug wird mit der entsprechenden Hilfsfunktion berechnet und zusammen mit dem Status `MIDDLE_GAME` zurückgegeben.

Hinweis:

Der Funktionsaufruf `board.is_game_over()` prüft, ob `checkmate`, `stalemate`, `insufficient material`, `seventyfive-move rule` oder `fivefold repetition` erfüllt sind und gibt in diesem Fall `True` zurück.

In [ ]:
from typing import Tuple, Union

class ChessAI(ChessAI):  # type: ignore
    def get_next_move(self: ChessAI, board: chess.Board) -> Tuple[Union[chess.Move, None], State]:
        '''Figures out the next possible move and switches between game states if necessary.'''   
        # Abort when fifty-move rule applies or game over
        if board.is_fifty_moves() or board.is_game_over():
            return None, State.FINISHED
        # If there is a move in the opening library find the best one
        elif self.state == State.OPENING_GAME and self.get_opening_game_reader().get(board):
            return self.get_next_opening_game_move(board), State.OPENING_GAME
        # If the endgame library provides a move use it
        elif len(board.piece_map()) <= 5:
            next_move = self.get_next_end_game_move(board)
            if next_move != None:
                return next_move, State.END_GAME
            return None, State.FINISHED
        # If neither the opening nor the endgame library has any moves available choose middle game move
        else:
            return self.get_next_middle_game_move(board), State.MIDDLE_GAME

## Ausführen des nächsten Zuges
Diese Funktion nimmt als Argument das aktuelle Board, führt einen Halbzug aus und protokolliert eventuelle Übergänge des Zustandes.

In [ ]:
class ChessAI(ChessAI):  # type: ignore
    state: State  # Declare for mypy

    def make_turn(self: ChessAI, board: chess.Board) -> None:
        """Makes a turn on the given board."""
        next_move, next_state = self.get_next_move(board)

        # Push next move or finish game
        if next_move:
            board.push(next_move)
        else:
            print(f"Finished with: {self.state} -> {State.FINISHED}")
            self.state = State.FINISHED
            # Close endgame tablebase reader
            self.get_end_game_reader().close()
            del self.end_game_reader
            return

        # Handle state transitions
        if next_state != self.state:
            # Close ployglot opening book to save memory if opening game has finished 
            if self.state == State.OPENING_GAME:
                self.get_opening_game_reader().close()
                del self.opening_game_reader
            self.state = next_state